In [ ]:
-- Podaj ilu jest pracowników płci męskiej, którzy urodzili się pomiędzy latami 1950-1960 (z wyłączeniem roku 1955), mieszkają w USA i w adresie mają frazę ‘Rd.’

SELECT COUNT(EmployeeID)
FROM Employees
WHERE (YEAR(BirthDate) BETWEEN 1950 AND 1954) OR (YEAR(BirthDate) BETWEEN 1956 AND 1960) AND Country LIKE 'USA' AND Address LIKE '%Rd.%'

In [ ]:
-- Wyświetl dane klientów (id, nazwy, adresy) którzy byli obsługiwani przez co najmniej dwóch pracowników i podaj dane tych pracowników.
-- Wykaz danych klientów uporządkuj rosnąco względem daty urodzenia

SELECT Customers.CustomerID, Customers.CompanyName, Customers.Address, Employees.EmployeeID
FROM Customers
JOIN Orders ON Orders.CustomerID= Customers.CustomerID
JOIN Employees ON Employees.EmployeeID=Orders.EmployeeID
GROUP BY Customers.CustomerID, Customers.CompanyName, Customers.Address, Employees.EmployeeID
HAVING COUNT(Orders.EmployeeID)>1
ORDER BY Customers.CustomerID


In [ ]:
-- Wyświetl wykaz zamówień, dla których opłata za przesyłkę mieściła się w przedziale 40%-60% 
-- średniej opłaty za przesyłkę wszystkich zamówień złożonych w miesiącach styczeń-marzec-maj.

SELECT OrderID
FROM Orders
WHERE Freight BETWEEN 0.4*(SELECT AVG(Freight)
                            FROM Orders
                            WHERE MONTH(OrderDate)=1 OR MONTH(OrderDate)=3 OR MONTH(OrderDate)=5) AND  0.6*(SELECT AVG(Freight)
                            FROM Orders
                            WHERE MONTH(OrderDate)=1 OR MONTH(OrderDate)=3 OR MONTH(OrderDate)=5)



In [ ]:
-- Podaj ilu jest pracowników, którzy urodzili się pomiędzy latami 1950-1953 lub 1955-1957 lub 1959-1962 i mieszkają w Londynie lub Seattle.
SELECT COUNT(EmployeeID)
FROM Employees
WHERE (YEAR(BirthDate) BETWEEN 1950 AND 1953) OR (YEAR(BirthDate) BETWEEN 1955 AND 1957) AND (City LIKE 'London' OR City LIKE 'Seattle')

In [ ]:
-- Wyświetl raport roczny sprzedaży w roku 1996 z podziałem na kwartały
-- (łączne kwoty, łączna liczba sztuk), uwzględniający zamówienia, dla których zanotowano liczbę sztuk powyżej średniej liczby sztuk w roku 1996
-- oraz wartość zamówienia powyżej średniej wartości zamówienia w roku 1996.

SELECT DATEPART(QUARTER, Orders.OrderDate) AS Quarters, SUM([Order Details].UnitPrice * [Order Details].Quantity*(1-[Order Details].Discount)), SUM(Quantity)
FROM Orders
JOIN [Order Details] ON [Order Details].OrderID = Orders.OrderID
WHERE YEAR(Orders.OrderDate) = 1996 AND [Order Details].Quantity> (SELECT AVG(Quantity)
                                                                    FROM [Order Details]
                                                                    JOIN Orders ON [Order Details].OrderID = Orders.OrderID
                                                                    WHERE YEAR(Orders.OrderDate) = 1996) AND [Order Details].UnitPrice * [Order Details].Quantity*(1-[Order Details].Discount)> (SELECT AVG([Order Details].UnitPrice * [Order Details].Quantity*(1-[Order Details].Discount))
                                                                    FROM [Order Details]
                                                                    JOIN Orders ON [Order Details].OrderID = Orders.OrderID
                                                                    WHERE YEAR(Orders.OrderDate) = 1996)
GROUP BY DATEPART(QUARTER, Orders.OrderDate)


In [ ]:
-- Wyświetl raport sprzedaży trzeciego kwartału roku 1998 z podziałem na miesiące (łączne kwoty, łączna liczba sztuk),
-- uwzględniający zamówienia, dla których zanotowano liczbę sztuk powyżej 30% średniej liczby sztuk w roku 1998 oraz
-- wartość zamówienia powyżej średniej wartości zamówienia w drugiej połowie roku 1998
SELECT MONTH(Orders.OrderDate) AS Mont, SUM([Order Details].UnitPrice * [Order Details].Quantity*(1-[Order Details].Discount)) AS TotalPrice, SUM([Order Details].Quantity) AS TotalQuantity
FROM Orders
LEFT OUTER JOIN [Order Details] ON [Order Details].OrderID=Orders.OrderID
WHERE  [Order Details].Quantity > 0.3*( SELECT AVG([Order Details].Quantity)
                                        FROM [Order Details]
                                        JOIN Orders ON Orders.OrderID=[Order Details].OrderID
                                        WHERE YEAR(Orders.OrderDate) = 1998)
        AND [Order Details].UnitPrice * [Order Details].Quantity*(1-[Order Details].Discount)>
                                      ( SELECT AVG([Order Details].UnitPrice * [Order Details].Quantity*(1-[Order Details].Discount))
                                        FROM [Order Details]
                                        JOIN Orders ON Orders.OrderID=[Order Details].OrderID
                                        WHERE YEAR(Orders.OrderDate) = 1998 AND MONTH(Orders.OrderDate) BETWEEN 7 AND 12 )
GROUP BY MONTH(Orders.OrderDate)
HAVING MONTH(Orders.OrderDate) BETWEEN 7 AND 9

In [ ]:
-- Wyświetl dane klientów (id, nazwy, adresy) klientów którzy byli obsługiwani tylko przez jednego pracownika lub dwóch pracowników. 
-- Podaj dane tych pracowników. Wykaz danych klientów uporządkuj alfabetycznie (rosnąco).

SELECT Customers.CustomerID, Customers.CompanyName, Customers.Address, Employees.EmployeeID
FROM Customers
JOIN Orders ON Orders.CustomerID= Customers.CustomerID
JOIN Employees ON Employees.EmployeeID=Orders.EmployeeID
GROUP BY Customers.CustomerID, Customers.CompanyName, Customers.Address, Employees.EmployeeID
HAVING COUNT(Orders.EmployeeID)<3
ORDER BY Customers.CustomerID

In [34]:
-- Wyświetl wykaz zamówień (numer zamówienia, nazwa klienta, nazwa przewoźnika, freight), 
-- dla których opłata za przesyłkę była mniejsza niż 70% średniej opłaty za przesyłkę wszystkich zamówień złożonych w miesiącach styczeń-luty-marzec.
-- Wykaz posortuj po nazwie przewoźnika


SELECT Orders.OrderID, Customers.CompanyName, Shippers.CompanyName, Orders.Freight
FROM Orders
JOIN Customers ON Customers.CustomerID = Orders.CustomerID
JOIN Shippers ON Shippers.ShipperID = Orders.ShipVia
WHERE Orders.Freight < 0.7*(SELECT AVG(Freight)
                            FROM Orders
                            WHERE MONTH(OrderDate)=1 OR MONTH(OrderDate)=3 OR MONTH(OrderDate)=5) 